# Expandir descripciones en textos de metadata de descripciones
Se expanden descripciones provenientes como arreglos de texto a un único texto unificado, así msimo, se genera un dataset separado para condensación de oraciones provenientes del corpus original

## Importes iniciales

In [11]:
%load_ext IPython.extensions.autoreload
%autoreload 2

The IPython.extensions.autoreload extension is already loaded. To reload it, use:
  %reload_ext IPython.extensions.autoreload


In [12]:
import sys
from pathlib import Path

def find_src_folder(current_path: Path, folder_name: str = 'src') -> Path:
    search_directories = [current_path] + list(current_path.parents)
    for parent in search_directories:
        if parent.name == folder_name:
            return parent.parent
    return current_path

src_path = find_src_folder(Path.cwd(), 'src')
sys.path.append(str(src_path))

In [24]:
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering

import matplotlib.pyplot as plt

import tensorflow as tf
import numpy as np

import pandas as pd

from matplotlib.ticker import FuncFormatter

import tensorflow_hub as hub

from sklearn.neighbors import NearestNeighbors

from langchain_ollama.embeddings import OllamaEmbeddings

from pyspark.sql import functions as F, types as T

from sklearn.cluster import DBSCAN

from src.utils import SparkUtils
from keybert import KeyBERT

In [14]:
spark_utils = SparkUtils('texts_expanded')

[SparkUtils] Stopping existing SparkSession...


In [19]:
spark = spark_utils.spark

In [20]:
meta_items = spark.read.parquet(spark_utils.path('meta_items'))

In [22]:
meta_items.limit(10).toPandas()

,title,main_category,features,description,average_rating,rating_number,price,store,parent_asin,categories,details,images
0,Accupressure Guide,Appstore for Android,[All the pressing point has been explained wit...,[Acupressure technique is very ancient and ver...,3.6,NaN,0.00,mAppsguru,B00VRPSGEO,[],"{""Release Date"":""2015"",""Date first listed on A...","[{""large"":""https://m.media-amazon.com/images/I..."
1,Ankylosaurus Fights Back - Smithsonian's Prehi...,Appstore for Android,[ENCOURAGE literacy skills with highlighted na...,[Join Ankylosaurus in this interactive book ap...,4.0,NaN,2.99,"Oceanhouse Media, Inc",B00NWQXXHQ,[],"{""Release Date"":""2014"",""Date first listed on A...","[{""large"":""https://m.media-amazon.com/images/I..."
2,Mahjong 2015,Appstore for Android,[Mahjong 2015 is a free solitaire matching gam...,[Mahjong 2015 is a free solitaire matching gam...,3.1,NaN,0.00,sophiathach,B00RFKP6AC,[],"{""Release Date"":""2014"",""Date first listed on A...","[{""large"":""https://m.media-amazon.com/images/I..."
3,Jewels Brick Breakout,Appstore for Android,"[Game Features:, - Intuitive touch controls wi...",[Jewels Brick Breakout is a glowing jewels bri...,4.2,NaN,0.00,Bad Chicken,B00SP2QU0E,[],"{""Release Date"":""2015"",""Date first listed on A...","[{""large"":""https://m.media-amazon.com/images/I..."
4,Traffic Police: Off-Road Cub,Appstore for Android,"[In this game you will find:, - Killer police ...",[Become the best road police officer in Cube C...,3.3,NaN,0.00,Dast 2 For Metro,B01DZIT64O,[],"{""Release Date"":""2016"",""Date first listed on A...","[{""large"":""https://m.media-amazon.com/images/I..."
5,Shadow Fighting Superhero War,Appstore for Android,"[Shadow combat super battle, Addicting gamepla...","[Let’s start to use different approach, dodge ...",3.3,NaN,0.00,3D Futuristic Games,B07F89XZF3,[],"{""Release Date"":""2018"",""Date first listed on A...","[{""large"":""https://m.media-amazon.com/images/I..."
6,My Aquarium - Hidden Object Challenge # 122,Appstore for Android,[100 Levels full of seek and find hidden objec...,[Did you search games for Hidden Object Games ...,3.0,NaN,0.00,PlayHOG,B00PY3WIEE,[],"{""Release Date"":""2014"",""Date first listed on A...","[{""large"":""https://m.media-amazon.com/images/I..."
7,Giant Surprise Petcage! Unlock Superhero Puppy,Appstore for Android,"[Latest surprise kit with pet goals., Include ...",[Love surprise opening and unboxing cute littl...,2.9,NaN,0.00,Surprise Unboxing Games,B08DNZVD7Q,[],"{""Release Date"":""2020"",""Date first listed on A...","[{""large"":""https://m.media-amazon.com/images/I..."
8,Ac Market tips,Appstore for Android,[We personally check and use the apps and play...,[Application made by fans for fans!Complete an...,3.3,NaN,0.00,Azeed,B076J3SJ4G,[],"{""Release Date"":""2017"",""Date first listed on A...","[{""large"":""https://m.media-amazon.com/images/I..."
9,Gem Shopping Network,Appstore for Android,"[Live Stream!, Call to shop, ask questions, an...",[Watch Gem Shopping Network Live!],3.8,816.0,0.00,Gem Shopping Network Inc.,B01GUEBBC0,[],"{""Release Date"":""2016"",""Date first listed on A...","[{""large"":""https://m.media-amazon.com/images/I..."


## Expandir descripción de oraciones para generación posterior de embeddings

In [29]:
# Separación de oraciones por descripción
meta_items_expanded_descriptions_sentences = meta_items.withColumn('description', F.explode('description'))
meta_items_expanded_descriptions_sentences =  meta_items_expanded_descriptions_sentences.withColumn(
    'description', F.split(F.col('description'), r"\.")
)
meta_items_expanded_descriptions_sentences = meta_items_expanded_descriptions_sentences.withColumn('description', F.explode('description'))

In [32]:
(
    meta_items_expanded_descriptions_sentences
        .write
        .format('delta')
        .mode('overwrite')
        .save(spark_utils.path('meta_items_description_sentences'))
)

## Unificar descripciones y características para extracción de palabras clave

In [31]:
# Unificación de descripción y características para extracción de palabras clave
meta_items_unified_descriptions_and_features = meta_items.withColumn(
    'description', F.concat_ws('.', F.col('description'))
).withColumn(
    'features', F.concat_ws('.', F.col('features'))
)

In [33]:
(
    meta_items_unified_descriptions_and_features
        .write
        .format('delta')
        .mode('overwrite')
        .save(spark_utils.path('meta_items_description_and_features_unified'))
)